<a href="https://colab.research.google.com/github/starcraftvs/testcamera/blob/master/tsdf_pca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install required modules
!pip install --quiet -U gdown
!pip install --quiet tensorflow-graphics

In [ ]:
#@title Useful imports

import numpy as np
import gdown
import h5py
import matplotlib.pyplot as plt
from skimage import measure
from tensorflow_graphics.notebooks import threejs_visualization

In [ ]:
#@title Download models
url = 'https://drive.google.com/uc?id=1Im74Wbj807iOLlWQpkTHIlislvCHsSVZ'
output = 'tsdf_pca_cars_bicycles.tar.gz'
gdown.cached_download(url, output, quiet=False, postprocess=gdown.extractall)

In [ ]:
#@title Helper functions

class TSDFPCA(object):
  def __init__(self, model_filepath):
    with h5py.File(model_filepath, 'r') as h5_file:
      self.basis = h5_file['tsdf_basis'].value
      self.mean = h5_file['tsdf_mean'].value
      self.thickness = np.squeeze(h5_file['thicknesses'].value)
      self.offset = - np.array(self.mean.shape) * self.thickness / 2.0

  def decode_to_tsdf(self, shape_code):
    return np.tensordot(self.basis, shape_code, axes=1) + self.mean

  def decode_to_mesh(self, shape_code):
    verts, faces, normals, values = measure.marching_cubes_lewiner(self.decode_to_tsdf(shape_code),level= 0, spacing=self.thickness)
    return {"vertices": verts + self.offset, "faces": faces}

  def code_size(self):
    return self.basis.shape[-1]

def visualize_mesh(mesh):
  camera = threejs_visualization.build_perspective_camera(
        field_of_view=30, position=(0.0, -1.5, 1.0))
  camera.lookAt ( 0.0, 0.0, 0.0)
  threejs_visualization.triangular_mesh_renderer([mesh], camera=camera, width=300, height=300)

In [ ]:
#@title Load and create a TSDFPCA model.
model = TSDFPCA('car_dpca20_128_ds1.h5')

In [ ]:
#@title Decode random code vectors and display decoded mesh.
code_size = model.code_size()
random_code = lambda : np.random.uniform(low=-40, high=40, size=code_size)

visualize_mesh(model.decode_to_mesh(np.zeros(code_size)))
visualize_mesh(model.decode_to_mesh(random_code()))
visualize_mesh(model.decode_to_mesh(random_code()))
visualize_mesh(model.decode_to_mesh(random_code()))
visualize_mesh(model.decode_to_mesh(random_code()))